In [1]:
import requests
import pandas as pd
import os
from datetime import datetime
import time

In [97]:
def find_class_in_block(block, class_name: str):
    class_title = block.find(class_name)
    return block[class_title:block.find('</a>', class_title)]


def find_obj_by_trigger(cl, trigger, end_trigger):
    len_tr = len(trigger)
    loc_tr = cl.find(trigger)
    return cl[loc_tr+len_tr+1: cl.find(end_trigger, loc_tr+len_tr+1)]


def string_obj(val):
    return f''' Продавец {val['Продавец']}
    Цена {val['Цена']}
    Бонусы {val['Бонусы']}
    Итого {val['Итог за вычетом бонуса']}
    Ссылка на карточку {val['Ссылка']}
    '''

In [199]:
def get_df_prices(ref_for_category):
    compare_prices = {}

    #ref_for_category = 'https://megamarket.ru/catalog/igrovye-pristavki-playstation/set-ps-5/'

    page = 1
    while True:
        print(page)
        search_result = requests.get(f'{ref_for_category}page-{page}')
        search_result.encoding = 'br'

        ost = search_result.text.split('<div class="sticky-element"><div class="catalog-listing__items catalog-listing__items_divider">')
        if len(ost) < 2:
            break
        ost_ids = ost[1]

        lists_count = ost_ids.split('<nav class="pager catalog-listing__pager">')[1]
        ost_ids = ost_ids.split('<nav class="pager catalog-listing__pager">')[0]
        ids = ost_ids.split('<div id=')[1:]

        for id_pr in ids:
            cl=find_class_in_block(id_pr, '<div class="item-title">')
            product_id = find_obj_by_trigger(cl, 'data-product-id=', '"')
            name = find_obj_by_trigger(cl, 'title=', '"')
            ref = 'https://megamarket.ru'+find_obj_by_trigger(cl, 'href=', '"')

            cl_merch = find_class_in_block(id_pr, '<div class="merchant-info__name"')
            saler = find_obj_by_trigger(cl_merch, '<div class="merchant-info__name"', '<')

            cl_price = find_class_in_block(id_pr, '<div class="item-money"')

            if len(cl_price) <2:
                break
            price = find_obj_by_trigger(cl_price, '<div class="item-price"><span', '<')
            price = int(price[:-2].replace(' ', ''))
            bonus = find_obj_by_trigger(cl_price, '<span class="bonus-amount"', '<')
            try:
                bonus = int(bonus.replace(' ', ''))
            except ValueError:
                bonus = 0

            compare_prices[product_id] = {'id': product_id,
                             'Наименование': name,
                             'Ссылка': ref,
                             'Продавец': saler,
                             'Цена': price,
                             'Бонусы': bonus,
                             'Итог за вычетом бонуса': price-bonus}
        page += 1

    res = pd.DataFrame().from_dict(compare_prices, orient='index').sort_values(by='Итог за вычетом бонуса')

    return f'''Топ 3 варианта:
    1. {string_obj(res.iloc[0])}
    2. {string_obj(res.iloc[1])}
    3. {string_obj(res.iloc[2])}'''

In [ ]:
get_df_prices(ref)

In [224]:
import telebot

API = '5077538054:AAFacmjTtKwlbz2m-FjWbG_WGxIgbkVVbR0'
bot = telebot.TeleBot(API)

In [225]:
@bot.message_handler(content_types=['text'])
def get_prices(ref):
    bot.reply_to(ref, get_df_prices(ref.text))

In [226]:
bot.infinity_polling()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


2023-12-01 15:29:37,801 (__init__.py:966 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2023-12-01 15:29:37,801 (__init__.py:968 MainThread) ERROR - TeleBot: "Break infinity polling"
